## Notebook-Dokumentation
- Lädt die Crash-CSV aus `raw_data/nypd` und prüft Umfang/Zeitraum/Borough-Abdeckung.
- Bereinigt Datum/Zeit-Spalten und zentralisierte Koordinatenverfügbarkeit.
- Basis-Deskriptoren (Verletzungen/Tote) als Qualitätssicherung.


Liest Dateipfad, setzt Polars-Config und zeigt Größe der Crash-CSV in MB.

In [15]:
from pathlib import Path
import polars as pl

pl.Config.set_tbl_rows(500)
DATA_PATH = Path("../../raw_data/nypd/Motor_Vehicle_Collisions_Crashes.csv")
DATA_PATH, round(DATA_PATH.stat().st_size / 1024**2, 1)


(PosixPath('../../raw_data/nypd/Motor_Vehicle_Collisions_Crashes.csv'), 533.1)

Definiert Schema, lädt CSV lazy, bereinigt Koordinaten und baut crash_datetime

In [16]:
pl.Config.set_fmt_str_lengths(120)

SCHEMA = {
    "CRASH DATE": pl.Utf8,
    "CRASH TIME": pl.Utf8,
    "BOROUGH": pl.Utf8,
    "ZIP CODE": pl.Utf8,
    "LATITUDE": pl.Utf8,
    "LONGITUDE": pl.Utf8,
    "LOCATION": pl.Utf8,
    "ON STREET NAME": pl.Utf8,
    "CROSS STREET NAME": pl.Utf8,
    "OFF STREET NAME": pl.Utf8,
    "NUMBER OF PERSONS INJURED": pl.Int64,
    "NUMBER OF PERSONS KILLED": pl.Int64,
    "NUMBER OF PEDESTRIANS INJURED": pl.Int64,
    "NUMBER OF PEDESTRIANS KILLED": pl.Int64,
    "NUMBER OF CYCLIST INJURED": pl.Int64,
    "NUMBER OF CYCLIST KILLED": pl.Int64,
    "NUMBER OF MOTORIST INJURED": pl.Int64,
    "NUMBER OF MOTORIST KILLED": pl.Int64,
    "CONTRIBUTING FACTOR VEHICLE 1": pl.Utf8,
    "CONTRIBUTING FACTOR VEHICLE 2": pl.Utf8,
    "CONTRIBUTING FACTOR VEHICLE 3": pl.Utf8,
    "CONTRIBUTING FACTOR VEHICLE 4": pl.Utf8,
    "CONTRIBUTING FACTOR VEHICLE 5": pl.Utf8,
    "COLLISION_ID": pl.Int64,
    "VEHICLE TYPE CODE 1": pl.Utf8,
    "VEHICLE TYPE CODE 2": pl.Utf8,
    "VEHICLE TYPE CODE 3": pl.Utf8,
    "VEHICLE TYPE CODE 4": pl.Utf8,
    "VEHICLE TYPE CODE 5": pl.Utf8,
}

scan = pl.scan_csv(
    DATA_PATH,
    schema=SCHEMA,
    infer_schema_length=2000,
    null_values=[""],
)
rename_map = {name: name.lower().replace(" ", "_") for name in scan.columns}
scan = scan.rename(rename_map)
scan = scan.with_columns(
    [
        pl.col("latitude").str.replace(",", ".").cast(pl.Float64, strict=False),
        pl.col("longitude").str.replace(",", ".").cast(pl.Float64, strict=False),
        pl.concat_str([pl.col("crash_date"), pl.col("crash_time")], separator=" ")
        .str.strptime(pl.Datetime, "%m/%d/%Y %H:%M", strict=False)
        .alias("crash_datetime"),
    ]
)


/var/folders/3d/gm4l_8mn4_59wxskbwlby73w0000gn/T/ipykernel_15090/3138952123.py:41: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  rename_map = {name: name.lower().replace(" ", "_") for name in scan.columns}


Aggregiert Basis-Kennzahlen (Zeilen, eindeutige collision_id, min/max crash_datetime)

In [22]:
overview = scan.select(
    [
        pl.len().alias("rows"),
        pl.col("collision_id").n_unique().alias("unique_collision_id"),
        pl.col("crash_datetime").min().alias("crash_min"),
        pl.col("crash_datetime").max().alias("crash_max"),
    ]
).collect()
overview


rows,unique_collision_id,crash_min,crash_max
u32,u32,datetime[μs],datetime[μs]
2233124,2233124,2012-07-01 00:05:00,2026-01-04 23:27:00


Zählt Null-Werte pro Spalte, berechnet Null-Prozent und sortiert

In [18]:
nulls = scan.select([pl.col(c).null_count().alias(c) for c in scan.columns]).collect()
total_rows = overview["rows"].item()

nulls_pd = nulls.to_pandas().T
nulls_pd.columns = ["nulls"]
nulls_pd["null_pct"] = (nulls_pd["nulls"] / total_rows * 100).round(2)
nulls_pd.sort_values("nulls", ascending=False)


/var/folders/3d/gm4l_8mn4_59wxskbwlby73w0000gn/T/ipykernel_15090/1428384794.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  nulls = scan.select([pl.col(c).null_count().alias(c) for c in scan.columns]).collect()


,nulls,null_pct
vehicle_type_code_5,2223320,99.56
contributing_factor_vehicle_5,2222999,99.55
vehicle_type_code_4,2197517,98.41
contributing_factor_vehicle_4,2196174,98.35
vehicle_type_code_3,2077785,93.04
contributing_factor_vehicle_3,2071468,92.76
off_street_name,1837960,82.30
cross_street_name,853472,38.22
zip_code,683041,30.59
borough,682762,30.57


Summiert Verletzte/Tote nach Personengruppen (Personen, Fußgänger, Radfahrer, Autofahrer)

In [19]:
severity = scan.select(
    [
        pl.sum("number_of_persons_injured").alias("injured_persons"),
        pl.sum("number_of_persons_killed").alias("killed_persons"),
        pl.sum("number_of_pedestrians_injured").alias("injured_pedestrians"),
        pl.sum("number_of_pedestrians_killed").alias("killed_pedestrians"),
        pl.sum("number_of_cyclist_injured").alias("injured_cyclists"),
        pl.sum("number_of_cyclist_killed").alias("killed_cyclists"),
        pl.sum("number_of_motorist_injured").alias("injured_motorists"),
        pl.sum("number_of_motorist_killed").alias("killed_motorists"),
    ]
).collect()
severity


injured_persons,killed_persons,injured_pedestrians,killed_pedestrians,injured_cyclists,killed_cyclists,injured_motorists,killed_motorists
i64,i64,i64,i64,i64,i64,i64,i64
735902,3547,133976,1761,65400,282,526065,1437


Zeigt Stichprobe von 5 Zeilen

In [21]:
scan.select(
    [
        "collision_id",
        "crash_datetime",
        "borough",
        "zip_code",
        "on_street_name",
        "cross_street_name",
        "number_of_persons_injured",
        "number_of_persons_killed",
    ]
).limit(5).collect()


collision_id,crash_datetime,borough,zip_code,on_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed
i64,datetime[μs],str,str,str,str,i64,i64
4455765,2021-09-11 02:39:00,null,null,"""WHITESTONE EXPRESSWAY""","""20 AVENUE""",2,0
4513547,2022-03-26 11:45:00,null,null,"""QUEENSBORO BRIDGE UPPER""",null,1,0
4675373,2023-11-01 01:29:00,"""BROOKLYN""","""11230""","""OCEAN PARKWAY""","""AVENUE K""",1,0
4541903,2022-06-29 06:55:00,null,null,"""THROGS NECK BRIDGE""",null,0,0
4566131,2022-09-21 13:21:00,null,null,"""BROOKLYN BRIDGE""",null,0,0
